![](https://cdn-va.niceshops.com/upload/image/banner/medium/default/17840.jpg)

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
from os import path
from PIL import Image

In [ ]:
df = pd.read_csv("../input/fast-food-restaurants-in-america/FastFoodRestaurants.csv")
df_counts = df.name.value_counts().to_frame().reset_index().rename(columns={"name": "ff_counts", "index": "name"})
df = df.merge(df_counts, on="name")

In [ ]:
fig = go.Figure(data=go.Scattergeo(
        lon = df['longitude'],
        lat = df['latitude'],
        text = df['name'],
        mode = 'markers',
        marker_color = df['ff_counts'],
        marker_symbol="circle-open",
        marker_colorscale="plasma",
        marker_showscale=True,
        marker_colorbar_title="Number of restaurants"))


fig.update_layout(
        title = 'All Fast Food Restaurants across USA',
        geo_scope='usa',
        title_x=0.5,
        font=dict(family="Times New Roman", size=18), font_color="#000000"
    )
iplot(fig)

In [ ]:
top_chain_df = df.drop_duplicates(subset={"name", "ff_counts"}).sort_values("ff_counts", ascending=False).head(10)

fig = px.bar(top_chain_df, x="name", y="ff_counts", color="name", text='ff_counts', 
             color_discrete_sequence = px.colors.sequential.Plasma_r,
             labels={'ff_counts':'number of restaurants in USA'})

fig.update_layout(
        title = 'Top restaurants',
        title_x=0.5,
        font=dict(family="Times New Roman", size=18), font_color="#000000"
    )

fig.update_traces(texttemplate='%{text:.2s} restaurants', textposition='outside')

fig.show()

In [ ]:
df_province_counts = df.province.value_counts().to_frame().reset_index().rename(columns={"province": "pr_counts", "index": "province"})

fig = go.Figure(data=go.Choropleth(
    locations=df_province_counts['province'], 
    z = df_province_counts['pr_counts'].astype(float), 
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Number of restaurants",
))

fig.update_layout(
    title_text = 'Restaurants State distribution',
    geo_scope='usa',
    title_x=0.5,
    font=dict(family="Times New Roman", size=18), font_color="#000000"# limite map scope to USA
)

fig.show()

In [ ]:
fig = go.Figure(data=go.Scattergeo(
        lon = df[df["ff_counts"]==1]['longitude'],
        lat = df[df["ff_counts"]==1]['latitude'],
        text = df[df["ff_counts"]==1]['name'],
        mode = 'markers',
        #marker_color = df[df["counts"]==1]['counts'],
        marker_symbol="square-open",
        marker_color="purple",
        hoverinfo="text"))
        #marker_showscale=True))


fig.update_layout(
        title = 'Non-Chain Restaurants across USA (exclusive)',
        geo_scope='usa',
        title_x=0.5,
        font=dict(family="Times New Roman", size=18), font_color="#000000"
    )
iplot(fig)

In [ ]:
plt.figure(figsize=(10,10))

d = '../input/burgerr/'

mask_b = np.array(Image.open(path.join(d, "kisspng-hamburger-button-junk-food-fast-food-chicken-sandw-burger-and-sandwich-5ad73c08479fc1.6174788015240550482934.png")))

text = df.name.unique()

cloud = WordCloud(background_color='white',colormap="Dark2", mask=mask_b
                ).generate(" ".join(text))

plt.title("Most common words in Fast Food name", size=20)
plt.axis("off")

plt.imshow(cloud, interpolation="bilinear")